In [ ]:
import os
import pandas as pd

In [ ]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

In [ ]:
xml_path = cfg.get("files", "dataset") + os.path.sep
dataframes = [pd.read_pickle(xml_path + f) for f in sorted(os.listdir(xml_path))]

In [ ]:
df_combined = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_combined.head()

In [ ]:
df_combined.to_pickle(cfg.get("files", "exchange") + os.path.sep + "01_dataframe_full.p")

In [1]:
"test"

'test'